# Objectifs du TP

Nous souhaitons mettre en oeuvre, *from scratch*, un arbre de décision pour la classification des données du jeu `Iris`. Plus spécifiquement, il va s'agir:

* D'observer les données, comprendre leur nature et les manipuler (éventuellement les transformer) pour les utiliser dans un arbre de décision (apprentissage et utilisation de l'arbre).
* De s'assurer que le fonctionnement d'un arbre de décision est compris, afin de pouvoir le mettre en oeuvre dans un programme.
* D'observer les résultats obtenus et de les mettre en perspective avec ce que l'on sait des données (à défaut de réaliser une évaluation plus poussée du modèle conçu).

# 1. Rappels conceptuels sur les arbres de décision

Le jeu de données `Iris` est constitué de 150 instances, chaque instance représentant un spécimen d'Iris décrit par quatre attributs et une classe. Techniquement, il y a donc cinq colonnes dans le fichier `iris.csv`.

Construire un arbre de décision sur des données consiste à placer les différentes attributes en tant que noeuds à l'intérieur de l'arbre. Les attributs seront placés selon leur *pouvoir discriminant*, c'est-à-dire la quantité d'information qu'ils apportent pour déterminer qu'une instance $\mathbf{x}$ est de classe $c$.

Dans ce TP, nous choisissons de construire un arbre de décision **binaire**. Chaque noeud placé dans l'arbre aura donc deux enfants, sauf s'il s'agit d'une feuille.

## Questions

1. Comment évalue-t-on le pouvoir discriminant d'un attribut ? Rappelez la formule permettant de réaliser le calcul.
2. Que peut-on dire du pouvoir discriminant placé à la racine de l'arbre, par rapport à celui des autres attributs ?
3. Quelles sont les informations contenues dans les noeuds de l'arbre ?
    1. Donc un arbre de décision permet de déterminer la classe d'une instance en réalisant une séquence de _____ ?
4. Les attributs du jeu de données que nous utilisons prennent des valeurs réelles. Comment devrons-nous gérer ces attributs ? (indiquez et détaillez plusieurs façons de procéder)

### Réponses

1. Le pouvoir discriminant d'un attribut est évalué par le gain d'information :
  Gain(S, A) = Entropie(S) - Σ(|Sv|/|S| × Entropie(Sv))
  où Entropie(S) = -Σ(pi × log2(pi))

2. L'attribut à la racine possède le plus grand gain d'information parmi tous les attributs disponibles.

3. Les nœuds contiennent :
  - L'attribut utilisé pour la décision
  - La valeur de division (split value)
  - Des pointeurs vers les nœuds enfants
  
    - Un arbre de décision détermine la classe d'une instance en réalisant une séquence de tests.

4. Gestion des attributs à valeurs réelles :
  - Seuil binaire : séparation optimale créant deux branches (< seuil et ≥ seuil)
  - Discrétisation en intervalles
  - Normalisation des valeurs
  - Arbres à branches multiples

# 2. Prise en main du code fourni

Vous avez à disposition une base de code dans le fichier `DecisionTree.py` (vous pourrez l'utiliser comme module dans le présent notebook; cf. partie "Mise en oeuvre"). Des fonctions y sont déjà mises en oeuvre:

1. `entropy(df, target_name)`
2. `attribute_gain(df, attribute, target)`
3. `best_attribute(df, attributes, target)`

La classe `DecisionTree` est partiellement mise en oeuvre.

## Questions

1. Reformulez, en pseudo-code, les trois fonctions listées ci-dessus
2. À quoi les variables d'instances de la classe `DecisionTree` correspondent-elles ? Comment seront-elles définies lors de la construction de l'arbre ?

# Reponses

## Pseudo-code des fonctions

### 1. `entropy(df, target_name)`
**But** : Calculer l'entropie d'un dataset pour un attribut cible donné.

```python
fonction entropy(df, target_name):
    initialiser une variable entropy à 0
    pour chaque valeur unique v dans df[target_name]:
        calculer la probabilité p de v dans df[target_name]
        entropy = entropy - p * log2(p)
    retourner entropy
```

---

### 2. `attribute_gain(df, attribute, target)`
**But** : Calculer le gain d'information d'un attribut par rapport à l'entropie de la cible.

```python
fonction attribute_gain(df, attribute, target):
    entropy_initiale = entropy(df, target)
    initialiser weighted_entropy à 0

    pour chaque valeur unique v dans df[attribute]:
        filtrer df pour obtenir subset où attribute = v
        calculer la probabilité p de subset par rapport à df
        weighted_entropy = weighted_entropy + (p * entropy(subset, target))

    retourner entropy_initiale - weighted_entropy
```

---

### 3. `best_attribute(df, attributes, target)`
**But** : Trouver l’attribut avec le plus grand gain d’information.

```python
fonction best_attribute(df, attributes, target):
    initialiser best_attr à None
    initialiser max_gain à -∞

    pour chaque attribut a dans attributes:
        gain = attribute_gain(df, a, target)
        si gain > max_gain:
            max_gain = gain
            best_attr = a

    retourner best_attr
```

---

## Variables d'instance de la classe `DecisionTree`

La classe `DecisionTree` représente un arbre de décision. Ses variables d'instance sont :

- **`attribute`** : L'attribut choisi pour diviser les données à ce niveau de l'arbre.
- **`value`** : La valeur de l'attribut pour ce nœud spécifique.
- **`children`** : Un dictionnaire contenant les sous-arbres (les branches issues de ce nœud).
- **`result`** : La classe majoritaire (utilisée si le nœud est une feuille).

### Définition lors de la construction de l’arbre
- L'algorithme sélectionne le meilleur attribut avec `best_attribute()`.
- Il crée un nœud avec cet attribut.
- Pour chaque valeur de l'attribut, il crée une branche et applique récursivement l'algorithme sur le sous-ensemble correspondant.
- Si tous les exemples ont la même classe ou si plus aucun attribut ne peut être sélectionné, il crée une feuille avec `result` contenant la classe majoritaire.



# 3. Mise en oeuvre

La cellule ci-dessous contient le code qui devra être exécuté lorsque vous aurez mis en oeuvre les deux fonctions vides dans le module `DecisionTree.py`.

## Travail à réaliser

1. Mettez en oeuvre la fonction `fit()` de la classe `DecisionTree`.
    1. Quels sont les cas de base de votre fonctions ?
    2. Que retournez-vous lorsque le cas de base est atteint ?
    3. Comment les branches de l'arbre sont-elles construites ? <br><br>
*Reponses :*
* 1. depth >= max_depth ou len(df[target].unique()) == 1 ou len(attributes) == 0 ou best_gain == 0
* 2. on retourne un nœud feuille (isLeaf=True)
* 3. L'arbre est construit récursivement en choisissant le meilleur attribut et sa valeur de division, en créant un nœud de décision, en divisant les données en deux partitions (< et ≥ à la valeur de division), puis en appelant fit() sur chaque partition pour former les sous-arbres gauche et droit.
    

2. Mettez en oeuvre la fonction `predict()` de la classe `DecisionTree`.
    1. Quel est le cas de base de votre fonction ?
    2. Que retournez-vous lorsque le cas de base est atteint ?
    3. Comment parcourez-vous votre arbre ? <br><br>
*Reponses :*
* 1. Le cas de base de la fonction : Le cas de base est atteint lorsqu'on arrive à un nœud feuille (quand isLeaf=True).
* 2. On retournes trois elements
    - La prédiction du nœud (contenant le décompte des classes)
    - La classe prédite (celle avec le plus grand nombre d'instances)
    - La proportion d'instances appartenant à la classe prédite (en pourcentage)
* 3. Le parcours se fait récursivement en comparant la valeur de l'attribut de l'instance avec la valeur de division du nœud :
    - Si la valeur de l'attribut est inférieure à la valeur de division, on parcourt la branche gauche
    - Sinon, on parcourt la branche droite

In [1]:
import pandas as pd
import numpy as np
import DecisionTree as dt

def test_DecisionTree(tree, test_data):
    """
    Tests the decision tree tree on test data
    Parameters :
        tree: fit decision tree (DecisionTree)
        test_data: set of instances from the dataset (dataframe)
    """
    #print(f"DEBUG: Entering test decision tree function")
    if tree == None: return None
    
    #print(f"DEBUG: tree not none")
    tree.print_tree()

    # Test the tree on a bunch of random instances
    for i in range(4):
        instance = test_data.sample()
        print(f"** Instance to predict: {instance[target]}\n")
        _, predicted_class, proportion = tree.predict(instance)
        print(f"Predicted class: {predicted_class} ({proportion})\n")
    return None

#print(f"DEBUG: Entering ")
# Load the data
df = pd.read_csv('iris.csv')

# Set a portion of data aside for testing
df_test  = df.sample(frac=0.15, random_state=42) # 22 instances
df_train = df.drop(df_test.index) # 128 instances

# Get attributes and target
target = 'class'
attributes = list(df_train.columns)[:-1] # attributes minus the target

# Instanciate and train a decision tree on df
#print("DEBUG: Avant création de l'arbre")
tree = dt.DecisionTree()
#print("DEBUG: Arbre créé, avant fit")
tree = tree.fit(df_train, target, attributes)
#print("DEBUG: Après fit, tree =", tree)

# Now, let's test that tree
test_DecisionTree(tree, df_test)

[Attribute: petal_length  Split value: 3.0]
> True
- Class Iris-setosa Count: 43
-
> False
-[Attribute: petal_length  Split value: 4.5]
-> True
-- Class Iris-versicolor Count: 26
--
-> False
--[Attribute: petal_width  Split value: 1.4]
--> True
--- Class Iris-versicolor Count: 1
---
--> False
---[Attribute: sepal_length  Split value: 5.4]
---> True
---- Class Iris-virginica Count: 1
----
---> False
----[Attribute: petal_length  Split value: 4.8]
----> True
----- Class Iris-versicolor Count: 9
-----
----> False
-----[Attribute: sepal_length  Split value: 5.9]
-----> True
------ Class Iris-virginica Count: 5
------
-----> False
------[Attribute: sepal_width  Split value: 2.5]
------> True
------- Class Iris-virginica Count: 1
-------
------> False
-------[Attribute: petal_width  Split value: 1.5]
-------> True
-------- Class Iris-virginica Count: 1
--------
-------> False
-------- Class Iris-virginica Count: 36
-------- Class Iris-versicolor Count: 5
--------
** Instance to predict: 12  

# Annexe - Résultats attendu

### Affichage de l'arbre
```
[Attribute: petal_length  Split value: 3.0]
> True
- Class Iris-setosa Count: 43
-
> False
-[Attribute: petal_width  Split value: 1.4]
-> True
-- Class Iris-versicolor Count: 23
--
-> False
--[Attribute: sepal_length  Split value: 5.2]
--> True
--- Class Iris-virginica Count: 1
---
--> False
---[Attribute: sepal_width  Split value: 2.5]
---> True
---- Class Iris-virginica Count: 1
----
---> False
---- Class Iris-virginica Count: 42
---- Class Iris-versicolor Count: 18
----
```

### Test sur 4 instances du jeu de test
```
** Instance to predict: 73    Iris-versicolor
Name: class, dtype: object

Node attribute: petal_length  Split value: 3.0
Instance's value for petal_length: 4.7
-Node attribute: petal_width  Split value: 1.4
-Instance's value for petal_width: 1.2
--> Node prediction:
 class
Iris-versicolor    23
Name: count, dtype: int64
Predicted class: Iris-versicolor (100.0)

** Instance to predict: 76    Iris-versicolor
Name: class, dtype: object

Node attribute: petal_length  Split value: 3.0
Instance's value for petal_length: 4.8
-Node attribute: petal_width  Split value: 1.4
-Instance's value for petal_width: 1.4
--Node attribute: sepal_length  Split value: 5.2
--Instance's value for sepal_length: 6.8
---Node attribute: sepal_width  Split value: 2.5
---Instance's value for sepal_width: 2.8
----> Node prediction:
 class
Iris-virginica     42
Iris-versicolor    18
Name: count, dtype: int64
Predicted class: Iris-virginica (70.0)

** Instance to predict: 110    Iris-virginica
Name: class, dtype: object

Node attribute: petal_length  Split value: 3.0
Instance's value for petal_length: 5.1
-Node attribute: petal_width  Split value: 1.4
-Instance's value for petal_width: 2.0
--Node attribute: sepal_length  Split value: 5.2
--Instance's value for sepal_length: 6.5
---Node attribute: sepal_width  Split value: 2.5
---Instance's value for sepal_width: 3.2
----> Node prediction:
 class
Iris-virginica     42
Iris-versicolor    18
Name: count, dtype: int64
Predicted class: Iris-virginica (70.0)

** Instance to predict: 73    Iris-versicolor
Name: class, dtype: object

Node attribute: petal_length  Split value: 3.0
Instance's value for petal_length: 4.7
-Node attribute: petal_width  Split value: 1.4
-Instance's value for petal_width: 1.2
--> Node prediction:
 class
Iris-versicolor    23
Name: count, dtype: int64
Predicted class: Iris-versicolor (100.0)
```